### 1. Install and Import Dependencies

In [3]:
!pip install mediapipe opencv-python
!pip install ultralytics
!pip install pybboxes

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip


In [1]:
import mediapipe as mp
import cv2
import numpy as np
from ultralytics import YOLO
import mediapipe as mp
import os
from ultralytics.utils.plotting import Annotator 


### 2. Load YOLO stuff

In [2]:
weights_loc = "/Users/taylor/runs/detect/train14/weights/best.pt"
yolo_model = YOLO(weights_loc)

### 3. Hand poses

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [ ]:

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(1)  # Adjust the device index as necessary

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        results = hands.process(image)
        canvas_for_yolo = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, c = frame.shape

        if results.multi_hand_landmarks:
            for handLMs in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    canvas_for_yolo, handLMs, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2)
                )

        width = canvas_for_yolo.shape[1]

        yolo_results = yolo_model.predict(frame)
        for r in yolo_results:
            annotator = Annotator(canvas_for_yolo)
            boxes = r.boxes
            for box in boxes:
                b = box.xyxy[0]
                mirrored_x_min = width - b[2]
                mirrored_x_max = width - b[0]
                mirrored_box = [mirrored_x_min, b[1], mirrored_x_max, b[3]]
                c = box.cls
                annotator.box_label(mirrored_box, "hand")

        cv2.imshow('Hand Skeleton', canvas_for_yolo)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()